In [2]:
import pandas as pd
import numpy as np
import random

In [7]:
recipe_data = pd.read_csv("../recipe.csv")
recipe_data.fillna("UNK", inplace=True)

cuisine_data = pd.read_csv("../cooking_data(cuisine).csv", sep=';')

chef_data = pd.read_csv("../chef.csv")

In [8]:
recipe_data.head()

,type,difficulty,name,description,prep_time,cook_time,cuisine_name,ingredient_name,portions,image_id
0,meal,3,Tavë Kosi,A traditional Albanian dish of baked lamb and ...,20,60,Albanian,lamb,1,0
1,dessert,4,Bakllasarem,A sweet pastry filled with a mixture of walnut...,30,45,Albanian,phyllo dough,1,0
2,meal,2,Fërgesë,"A hearty dish of peppers, tomatoes, and cottag...",15,30,Albanian,red bell pepper,1,0
3,meal,3,Speca të mbushur,"Bell peppers stuffed with rice, herbs, and sea...",20,40,Albanian,red bell pepper,1,0
4,meal,3,Byrek me Spinaq,"A savory pie made with phyllo dough, filled wi...",20,30,Albanian,phyllo dough,1,0


In [11]:
# Cuisines generator

cuisines = {
    "name":[],
    "image_id":[]
}

for row in cuisine_data.itertuples(index=True, name='Pandas'):
    cuisines["name"].append(getattr(row, "name"))
    if(len(cuisines["image_id"]) != 0):
        cuisines["image_id"].append(cuisines["image_id"][-1]+1)
    else:
        cuisines["image_id"].append(1)
    
for row in recipe_data.itertuples(index=True, name='Pandas'):
    cuisine_name = getattr(row, "cuisine_name")
    if not(cuisine_name in cuisines["name"]):
        cuisines["name"].append(cuisine_name)
        cuisines["image_id"].append(cuisines["image_id"][-1]+1)

print(cuisines)
pd.DataFrame(cuisines).to_csv("cuisine.csv", index=False)

{'name': ['Greek', 'French', 'Indian', 'Italian', 'Japanese', 'Chinese', 'Mexican', 'German', 'Korean', 'Turkish', 'Albanian', 'American', 'Bulgarian', 'Russian'], 'image_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}


In [14]:
# Modify recipes

# Change column names
new_recipe = recipe_data.rename(columns={'cook_time': 'cook_time_m', 'prep_time': 'prep_time_m'})

# Change type names
new_recipe['type'] = np.where(new_recipe['type'] == 'meal', 'Cooking', 'Pastry')

# Change cuisine names to include all cuisines
rec_len = len(new_recipe['cuisine_name'])
new_cuisine_name = { 'cuisine_name': [] }

for i in range(0, rec_len):
    new_cuisine_name['cuisine_name'].append(cuisines['name'][i % len(cuisines['name'])])

new_recipe.update(pd.DataFrame(new_cuisine_name))
new_recipe.to_csv("recipe.csv", index=False)

In [20]:
# chef_has_cuisine generator

chef_has_cuisine = {
    "chef_id":[],
    "cuisine_name":[]
}

chef_ids = []
for row in chef_data.itertuples(index=True, name='Pandas'):
    chef_ids.append(getattr(row, 'id'))

for id in chef_ids:
    s = random.randint(1, 3)
    selection = random.sample(cuisines["name"], s)
    
    for i in range(0, s):
        chef_has_cuisine["chef_id"].append(id)       
        chef_has_cuisine["cuisine_name"].append(selection[i])

pd.DataFrame(chef_has_cuisine).to_csv("chef_has_cuisine.csv", index=False)

In [18]:
nr_data = pd.read_csv("recipe.csv")
recipe_cuisine = {}
for row in nr_data.itertuples(index=True, name='Pandas'):
    recipe_cuisine[getattr(row, 'name')] = getattr(row, 'cuisine_name')

In [33]:
step_data = pd.read_csv("../step.csv")

# remove ukrainian recipe steps
step_data = step_data[step_data.recipe_cuisine_name != 'Ukrainian']
#print(step_data.recipe_cuisine_name)

new_step_cuisine = {'recipe_cuisine_name':[]}
for row in step_data.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    #print(recipe_name)
    new_step_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])
step_data.update(new_step_cuisine)
step_data.to_csv("step.csv", index=False)

In [35]:
# change _has_equipment cuisines and create equipment
he = pd.read_csv("../recipe_has_equipment.csv")
he = he[he.recipe_cuisine_name != 'Ukrainian']

new_he_cuisine = {'recipe_cuisine_name':[]}

equipment = {
    "name":[],
    "manual":[],
    "image_id":[]
}

for row in he.itertuples(index=True, name='Pandas'):
    recipe_name = getattr(row, 'recipe_name')
    new_he_cuisine['recipe_cuisine_name'].append(recipe_cuisine[recipe_name])

    eq = getattr(row, 'equipment')
    if not(eq in equipment["name"]):
        equipment["name"].append(eq)
        equipment["manual"].append("used for cooking")
        equipment["image_id"].append(0)

he.update(new_he_cuisine)
he.to_csv("recipe_has_equipment.csv", index=False)

pd.DataFrame(equipment).to_csv("equipment.csv", index=False)